In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 设备选择
device = torch.device(
    "mps"
    if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available() else "cpu"
)
# device = torch.device("cpu")
print(f"Using device: {device}")

# 1. 数据预处理
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

# 2. 加载 Fashion-MNIST 数据集
batch_size = 64
train_dataset = torchvision.datasets.FashionMNIST(
    root="../data", train=True, transform=transform, download=True
)
test_dataset = torchvision.datasets.FashionMNIST(
    root="../data", train=False, transform=transform, download=True
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 3. 定义 LeNet 网络
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)  # 保持 28x28
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # 下采样变 14x14
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)  # 10x10
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # (batch, 6, 14, 14)
        x = self.pool(torch.relu(self.conv2(x)))  # (batch, 16, 5, 5)
        x = torch.flatten(x, 1)  # 展平
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # 10 分类
        return x


# 4. 创建模型
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 训练
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 6. 测试
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Using device: mps
Epoch [1/10], Loss: 0.5590
Epoch [2/10], Loss: 0.3581
Epoch [3/10], Loss: 0.3106
Epoch [4/10], Loss: 0.2835
Epoch [5/10], Loss: 0.2642
Epoch [6/10], Loss: 0.2477
Epoch [7/10], Loss: 0.2324
Epoch [8/10], Loss: 0.2214
Epoch [9/10], Loss: 0.2077
Epoch [10/10], Loss: 0.1949
Test Accuracy: 89.57%
